In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

df = pd.read_csv('cop_data.csv', sep=';', decimal=',')
df['M_HP'] = round(df['M_HP'] / 60,2)
df['LWT'] = df['LWT'] + 273
df['T_OA'] = df['T_OA'] + 273
df

,T_OA,T_sup_HP,COP_real
0,248,303,2.08
1,253,303,2.26
2,258,303,2.45
3,263,303,3.12
4,268,303,3.30


# Predicting COP with a linear model

In [2]:
# Declare the model
mod = smf.ols(formula='COP ~ M_HP + LWT + T_OA', data=df)

# Fit the model (finds the optimal coefficients, adding a random seed ensures consistency)
np.random.seed(2) 
res = mod.fit()

# Print the summary output
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                    COP   R-squared:                       0.958
Model:                            OLS   Adj. R-squared:                  0.956
Method:                 Least Squares   F-statistic:                     560.7
Date:                Sun, 22 Oct 2023   Prob (F-statistic):           8.81e-51
Time:                        20:23:38   Log-Likelihood:                -25.795
No. Observations:                  78   AIC:                             59.59
Df Residuals:                      74   BIC:                             69.02
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.7789      2.945     -0.264      0.7

In [3]:
def cop(massf, supT, outT):
    return res.params.Intercept + res.params.M_HP*massf + res.params.LWT*supT + res.params.T_OA*outT

# Get all the predicted COPs in a list
COP_list = []
for i in range(len(df['T_OA'])):
    COP_list.append(round(cop(df['M_HP'][i],df['LWT'][i],df['T_OA'][i]),2))

# Get a list of all the real COPs
COP_list_real = list(df.COP)

# Compare with the real COPs
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(COP_list_real, COP_list)
print("MSE = {}".format(round(mse,3)))
print("sqrt(MSE) = {}".format(round(np.sqrt(mse),3)))

# This is just for printing for excel
# COP_list = [str(x).replace(".",",") for x in COP_list]
# for i in range(len(df['T_OA'])):
#    print(COP_list[i])

MSE = 0.113
sqrt(MSE) = 0.336


In [4]:
print("Intercept: {}".format(round(res.params.Intercept,4)))
print("M_HP coeff: {}".format(round(res.params.M_HP,4)))
print("LWT coeff: {}".format(round(res.params.LWT,4)))
print("T_OA coeff: {}".format(round(res.params.T_OA,4)))

Intercept: -0.7789
M_HP coeff: -0.0726
LWT coeff: -0.0658
T_OA coeff: 0.0911


In [5]:
print(cop(0.76, 284.81, 293))

7.137510880573103


# Predicting COP with a quadratic model

In [6]:
# Declare the model
mod = smf.ols(formula='COP ~ M_HP + LWT + T_OA + I(M_HP**2) + I(LWT**2) + I(T_OA**2)', data=df)

# Fit the model (finds the optimal coefficients, adding a random seed ensures consistency)
np.random.seed(2) 
res = mod.fit()

# Print the summary output
# print(res.summary())

In [7]:
def cop(massf, supT, outT):
    return res.params.Intercept + res.params.M_HP*massf + res.params.LWT*supT \
    + res.params.T_OA*outT \
    + res.params['I(M_HP ** 2)']*(massf**2) \
    + res.params['I(LWT ** 2)']*(supT**2) \
    + res.params['I(T_OA ** 2)']*(outT**2)

# Get all the predicted COPs in a list
COP_list = []
for i in range(len(df['T_OA'])):
    COP_list.append(round(cop(df['M_HP'][i],df['LWT'][i],df['T_OA'][i]),2))
    
# Get a list of all the real COPs
COP_list_real = list(df.COP)

# Compare with the real COPs
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(COP_list_real, COP_list)
print("MSE = {}".format(round(mse,3)))
print("sqrt(MSE) = {}".format(round(np.sqrt(mse),3)))

# This is just for printing for excel

#COP_list = [str(x).replace(".",",") for x in COP_list]
#for i in range(len(df['T_OA'])):
#    print(COP_list[i])

MSE = 0.088
sqrt(MSE) = 0.297


In [8]:
# Declare the model
mod = smf.ols(formula='COP ~ M_HP * LWT + T_OA', data=df)

# Fit the model (finds the optimal coefficients, adding a random seed ensures consistency)
np.random.seed(2) 
res = mod.fit()

# Print the summary output
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                    COP   R-squared:                       0.958
Model:                            OLS   Adj. R-squared:                  0.956
Method:                 Least Squares   F-statistic:                     414.8
Date:                Sun, 22 Oct 2023   Prob (F-statistic):           2.27e-49
Time:                        20:23:39   Log-Likelihood:                -25.794
No. Observations:                  78   AIC:                             61.59
Df Residuals:                      73   BIC:                             73.37
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.1386     24.027     -0.006      0.9

In [9]:
def cop(massf, supT, outT):
    return res.params.Intercept + res.params.M_HP*massf + res.params.LWT*supT \
    + res.params.T_OA*outT + res.params['M_HP:LWT']*(massf*supT)

# Get all the predicted COPs in a list
COP_list = []
for i in range(len(df['T_OA'])):
    COP_list.append(round(cop(df['M_HP'][i],df['LWT'][i],df['T_OA'][i]),2))
    
# Get a list of all the real COPs
COP_list_real = list(df.COP)

# Compare with the real COPs
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(COP_list_real, COP_list)
print("MSE = {}".format(round(mse,3)))
print("sqrt(MSE) = {}".format(round(np.sqrt(mse),3)))

# This is just for printing for excel

#COP_list = [str(x).replace(".",",") for x in COP_list]
#for i in range(len(df['T_OA'])):
#    print(COP_list[i])

MSE = 0.113
sqrt(MSE) = 0.337


## Get $m_{HP}$ from $T_{sup,HP}$

In [10]:
# Get the data
df2 = pd.read_csv('mhp_data.csv', sep=';', decimal=',')

# Declare the model
mod = smf.ols(formula='m_HP ~ T_sup', data=df2)

# Fit the model (finds the optimal coefficients, adding a random seed ensures consistency)
np.random.seed(2) 
res = mod.fit()

# Print the summary output
# print(res.summary())

def mass_HP(T_HP):
    return res.params.Intercept + res.params.T_sup*T_HP

In [11]:
print(mass_HP(40))

42.10714285714284
